In [1]:
import json
import os
import time
import pandas as pd
import random
import time
import pickle

import numpy as np
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout

from keras.backend import clear_session
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
K.set_session(session)

Using TensorFlow backend.


In [2]:

with open("bb_train<20.json") as f:
    all_data=json.load(f)
print('num samples: ',len(all_data))


#cant be too large, will trigger a memory error later on. Failed at 1 Mill
#np.random.shuffle(all_data)

data = all_data[:1000000]





#Need to optimize this later, replace use np arrays 
#instead of python lists

#top_k = 10000

t = Tokenizer(oov_token='<unk>')
t.fit_on_texts(data)


with open('bb-v3.1-tok.pickle', 'wb') as handle:
    pickle.dump(t, handle, protocol=pickle.HIGHEST_PROTOCOL)


start = time.time()
seqs = [t.texts_to_sequences(x) for x in data]
print('elaspted text->int seqs: ',time.time() - start)

seq_data = list()
for seq in seqs:
    flat = [num for sublist in seq for num in sublist]
    seq_data.append(flat)
    
print('total seq construction time: ',time.time() - start)

num samples:  55873682
elaspted text->int seqs:  46.21088671684265
total seq construction time:  48.28778028488159


Need to optimize this later, use np arrays instead of python lists


For reference, seqs took 57s to go through 1 mill samples

In [3]:
max_len = max([len(seq) for seq in seq_data])

sequences = pad_sequences(seq_data, maxlen=max_len, padding='pre')

print('Max Seq Len: %d' % max_len)

sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
vocab_sz = len(t.word_index)+1
print('vocab sz: ',vocab_sz)

#y_cat = to_categorical(y, num_classes=vocab_sz) #1-hot encoding 

Max Seq Len: 19
vocab sz:  41031


In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.20, random_state=7)

#NN definition
model = Sequential()
model.add(Embedding(vocab_sz, 300, input_length=max_len-1))
model.add(LSTM(500))



model.add(Dense(vocab_sz, activation='softmax'))
print(model.summary())


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 18, 300)           12309300  
_________________________________________________________________
lstm_1 (LSTM)                (None, 500)               1602000   
_________________________________________________________________
dense_1 (Dense)              (None, 41031)             20556531  
Total params: 34,467,831
Trainable params: 34,467,831
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

#save model structure as json

model_json = model.to_json()
with open("models/huffpo-model-v3.1.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:

model_json = model.to_json()
with open("models/bb-model-v3.1json", "w") as json_file:
    json_file.write(model_json)

fpath = "weights/bb-v3.1/best-weights.hdf5"
checkpoint = ModelCheckpoint(fpath, monitor='val_acc', verbose=2, save_best_only=True, mode='max')


BS = 100
tb = TensorBoard(log_dir="tensorboard-logs/{}".format('bb-modelv3.1'))
callback_lst = [checkpoint, tb]
#steps_per_epoch is num of batches that make up 1 epoch, defaults to size of train set
model.fit(X,y,batch_size=BS, validation_split=.005, epochs=200, callbacks=callback_lst, verbose=1)



Instructions for updating:
Use tf.cast instead.
Train on 995000 samples, validate on 5000 samples
Epoch 1/200
995000/995000 [==============================] - 542s 545us/step - loss: 2.6106 - acc: 0.2412 - val_loss: 2.5350 - val_acc: 0.2540

Epoch 00001: val_acc improved from -inf to 0.25400, saving model to weights/bb-v3.1/best-weights.hdf5
Epoch 2/200
995000/995000 [==============================] - 537s 540us/step - loss: 2.5656 - acc: 0.2436 - val_loss: 2.5147 - val_acc: 0.2584

Epoch 00002: val_acc improved from 0.25400 to 0.25840, saving model to weights/bb-v3.1/best-weights.hdf5
Epoch 3/200
995000/995000 [==============================] - 540s 542us/step - loss: 2.5641 - acc: 0.2440 - val_loss: 2.5304 - val_acc: 0.2394

Epoch 00003: val_acc did not improve from 0.25840
Epoch 4/200
995000/995000 [==============================] - 539s 541us/step - loss: 2.5637 - acc: 0.2438 - val_loss: 2.5269 - val_acc: 0.2394

Epoch 00004: val_acc did not improve from 0.25840
Epoch 5/200
995000/